# TP1

Autores: Afonso FFerreira pg52669, Tiago Rodrigues pg52705

### Exercício 1

Use a package Cryptography   e  o package ascon para  criar um comunicação privada assíncrona em modo  “Lightweight Cryptography” entre um agente Emitter e um agente Receiver que cubra os seguintes aspectos:

- a) Autenticação do criptograma e dos metadados (associated data) usando Ascon em modo de cifra.

- b) As chaves de cifra, autenticação  e  os “nounces” são gerados por um gerador pseudo aleatório (PRG)  usando o Ascon em modo XOF. As diferentes chaves para inicialização do PRG são inputs do emissor e do receptor.

- c) Para implementar a comunicação cliente-servidor use o package python asyncio

In [16]:
from ascon import encrypt, decrypt, hash
import asyncio
import os

def emitter(plaintext, key, nonce, associateddata):
    cifra = encrypt(key, nonce, associateddata, plaintext, variant="Ascon-128")
    #criptograma = cifra[:-16]
    #tag = cifra[-16:]
    return cifra#, criptograma, tag

def receiver(key, nonce, associateddata, cifra):
    receivedPlainText = decrypt(key, nonce, associateddata, cifra, variant="Ascon-128")
    if receivedPlainText == None: print("Verification failed :(")
    return receivedPlainText

def pseudoRandomGenerator():
    keySeed = os.urandom(16)
    key = hash(keySeed, variant="Ascon-Xof", hashlength=16)

    nonceSeed = os.urandom(16)
    nonce = hash(nonceSeed, variant="Ascon-Xof", hashlength=16)
    return key, nonce

# Server
async def handle_echo(reader, writer):
    data = await reader.read(100)
    message = data

    addr = writer.get_extra_info('peername')

    print(f"Server has received {message} from client {addr}")

    print("Decrypting...")
    recebido = receiver(key, nonce, associateddata, message)

    print(f"Server has sent: {recebido}")
    writer.write(data)
    await writer.drain()

    print("Closing the server connection")
    writer.close()
    

async def main():
    server = await asyncio.start_server(
        handle_echo, '127.0.0.1', 8889)

    addr = server.sockets[0].getsockname()
    print(f'Serving on {addr}')

# Client
async def tcp_echo_client():
    await asyncio.sleep(3)
    reader, writer = await asyncio.open_connection(
        '127.0.0.1', 8889)

    print('Encrypting...')
    cifra = emitter(plaintext, key, nonce, associateddata)

    print(f'Client has sent: {cifra}')
    writer.write(cifra)

    data = await reader.read(100)
    print(f'Client has received: {data}')

    print('Closing the client connection')
    writer.close()

async def run_client_and_server():
    await asyncio.gather(tcp_echo_client(), main())


plaintext = b"Anacleto manda mensagem a Bernardina"
key, nonce = pseudoRandomGenerator()
associateddata = b"ASCON"

await run_client_and_server()

Serving on ('127.0.0.1', 8889)
Encrypting...
Client has sent: b'J\xd6)\xb4V\x06R\xbb\xa4t\xe6=\xcc\xb7\x84\x0e\xea\xec\x00\x04\xcb\xc9\xb4\x8dX\x7fF\x82}6~\xb8~-\x03\xc3\xf7\xa1\xf3\x86\xc0+[h_\xc4\x0b\x04b\x8b\xe8\xfa'
Server has received b'J\xd6)\xb4V\x06R\xbb\xa4t\xe6=\xcc\xb7\x84\x0e\xea\xec\x00\x04\xcb\xc9\xb4\x8dX\x7fF\x82}6~\xb8~-\x03\xc3\xf7\xa1\xf3\x86\xc0+[h_\xc4\x0b\x04b\x8b\xe8\xfa' from client ('127.0.0.1', 45258)
Decrypting...
Server has sent: b'Anacleto manda mensagem a Bernardina'
Closing the server connection
Client has received: b'J\xd6)\xb4V\x06R\xbb\xa4t\xe6=\xcc\xb7\x84\x0e\xea\xec\x00\x04\xcb\xc9\xb4\x8dX\x7fF\x82}6~\xb8~-\x03\xc3\xf7\xa1\xf3\x86\xc0+[h_\xc4\x0b\x04b\x8b\xe8\xfa'
Closing the client connection


### Exercício 2


Use o “package” Cryptography para
- a) Implementar uma AEAD com “Tweakable Block Ciphers” conforme está descrito na última secção do texto +Capítulo 1: Primitivas Criptográficas Básicas.  A cifra por blocos primitiva, usada para gerar a “tweakable block cipher”, é o AES-256 ou o ChaCha20.
- b) Use esta cifra para construir um canal privado de informação assíncrona com acordo de chaves feito com “X448 key exchange” e “Ed448 Signing&Verification” para autenticação  dos agentes. Deve incluir uma fase de confirmação da chave acordada.

In [20]:
import os
from cryptography.hazmat.primitives.ciphers.aead import ChaCha20Poly1305

data = b"a secret message"

aad = b"authenticated but unencrypted data"

key = ChaCha20Poly1305.generate_key()

chacha = ChaCha20Poly1305(key)

nonce = os.urandom(12)

ct = chacha.encrypt(nonce, data, aad)

original = chacha.decrypt(nonce, ct, aad)
original

b'a secret message'